In [32]:
import pybamm
import numpy as np
from SALib.sample import saltelli
from SALib.analyze import sobol
import matplotlib.pyplot as plt
import pandas as pd

In [33]:
df = pd.read_csv("filtered_data.csv")

In [34]:
new_time = np.arange(0, len(df['Testtime [s]']))
df['Testtime [s]'] = new_time
drive_cycle = df.to_numpy()
time_data = drive_cycle[:, 1]  # Time data in seconds
current_data = drive_cycle[:, 3]  # Current data in amperes

In [35]:
param = pybamm.ParameterValues("OKane2022")
print(param)

{'Ambient temperature [K]': 298.15,
 'Boltzmann constant [J.K-1]': 1.380649e-23,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.2594,
 'Cell cooling surface area [m2]': 0.00531,
 'Cell thermal expansion coefficient [m.K-1]': 1.1e-06,
 'Cell volume [m3]': 2.42e-05,
 'Contact resistance [Ohm]': 0,
 'Current function [A]': 5.0,
 'Dead lithium decay constant [s-1]': 1e-06,
 'Dead lithium decay rate [s-1]': <function SEI_limited_dead_lithium_OKane2022 at 0x13cf26b60>,
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Electrode height [m]': 0.065,
 'Electrode width [m]': 1.58,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Nyman2008_arrhenius at 0x13cf27240>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Nyman2008_arrhenius at 0x13cf271a0>,
 'Electron charge [C]': 1.602176634e-19,
 'Exchange-current density for plating [A.m-2]': <function plating_exchange_curr

In [21]:
param.search("coefficient")

Cell thermal expansion coefficient [m.K-1]	1.1e-06
Lithium plating transfer coefficient	0.65
Negative electrode Bruggeman coefficient (electrode)	1.5
Negative electrode Bruggeman coefficient (electrolyte)	1.5
Negative electrode charge transfer coefficient	0.5
Positive electrode Bruggeman coefficient (electrode)	1.5
Positive electrode Bruggeman coefficient (electrolyte)	1.5
Positive electrode charge transfer coefficient	0.5
Separator Bruggeman coefficient (electrolyte)	1.5
Total heat transfer coefficient [W.m-2.K-1]	10.0


In [46]:
def battery_model(parameters):
    # 创建DFN模型
    model = pybamm.lithium_ion.DFN()
    
    # 加载"OKane2022"参数集
    param = pybamm.ParameterValues("OKane2022")
    
    # 更新参数
    param.update({
        "Negative electrode thickness [m]": parameters[0], #l_neg :Anode thickness 35-79e-6
        "Positive electrode thickness [m]": parameters[1], #l_pos :Cathode thickness 35-79e-6
        "Separator thickness [m]": parameters[2], #l_sep :Separator thickness 10-30e-6
        "Negative electrode active material volume fraction": parameters[3], 
        #epsilon_f_neg: Anode active material volume fraction 0.4-0.5
        "Positive electrode active material volume fraction": parameters[4],
        #epsilon_f_pos: Cathode active material volume fraction 0.35-0.5
        "Negative particle radius [m]": parameters[5], #Rp_neg: 1-11e-6
        "Positive particle radius [m]": parameters[6], #Rp_pos: 1-11e-6
        "Initial concentration in electrolyte [mol.m-3]": parameters[7], #c0: 1000-1200
        
    })
    # 创建Casadi求解器并设置选项
    solver = pybamm.CasadiSolver(dt_max=0.1, max_num_steps=1000)
    
    # 创建仿真
    sim = pybamm.Simulation(model, parameter_values=param, solver=solver)
    
    # 运行仿真
    try:
        sim.solve([0, 3600])  # 模拟时间范围，单位为秒
    except pybamm.SolverError as e:
        print(f"Solver failed for parameters: {parameters} with error: {e}")
        return np.nan  # 返回NaN以继续处理其他参数组合
    
    # 提取仿真结果
    solution = sim.solution
    voltage = solution["Terminal voltage [V]"].entries
    
    # 返回仿真结束时的电压
    return voltage[-1]


In [47]:
# 定义参数空间
problem = {
    'num_vars': 8,
    'names': ['Negative electrode thickness [m]', 
              'Positive electrode thickness [m]', 
              'Separator thickness [m]',
              'Negative electrode active material volume fraction',
              'Positive electrode active material volume fraction',
              'Negative particle radius [m]',
              'Positive particle radius [m]',
              'Initial concentration in electrolyte [mol.m-3]',],
    'bounds': [[35e-6, 79e-6],  # Negative electrode thickness
               [35e-6, 79e-6],  # Positive electrode thickness
               [10e-6, 30e-6],  # Separator thickness
               [0.4, 0.5],      # Negative electrode active material volume fraction
               [0.35, 0.5],     # Positive electrode active material volume fraction
               [1e-6, 11e-6],   # Negative particle radius
               [1e-6, 11e-6],   # Positive particle radius
               [1e3, 1.2e3]     # Initial concentration in electrolyte
               ]       
}

In [48]:
# 生成样本
param_values = saltelli.sample(problem, 1024)

In [49]:
print(param_values)

[[3.50644531e-05 5.15644531e-05 1.89550781e-05 ... 9.44238281e-06
  3.41699219e-06 1.11748047e+03]
 [6.56582031e-05 5.15644531e-05 1.89550781e-05 ... 9.44238281e-06
  3.41699219e-06 1.11748047e+03]
 [3.50644531e-05 6.45410156e-05 1.89550781e-05 ... 9.44238281e-06
  3.41699219e-06 1.11748047e+03]
 ...
 [7.52402344e-05 5.46152344e-05 1.99707031e-05 ... 1.08779297e-05
  2.17675781e-06 1.13564453e+03]
 [7.52402344e-05 5.46152344e-05 1.99707031e-05 ... 1.08779297e-05
  5.88769531e-06 1.00634766e+03]
 [7.52402344e-05 5.46152344e-05 1.99707031e-05 ... 1.08779297e-05
  5.88769531e-06 1.13564453e+03]]


In [50]:
# 运行模型并获取结果
Y = []

for i, params in enumerate(param_values):
    try:
        result = battery_model(params)
        Y.append(result)
    except Exception as e:
        print(f"Error for sample {i} with parameters {params}: {e}")
        Y.append(np.nan)  # 返回NaN以继续处理其他参数组合

Y = np.array(Y)

Error for sample 0 with parameters [3.50644531e-05 5.15644531e-05 1.89550781e-05 4.48681641e-01
 4.33569336e-01 9.44238281e-06 3.41699219e-06 1.11748047e+03]: CasadiSolver.__init__() got an unexpected keyword argument 'max_num_steps'
Error for sample 1 with parameters [6.56582031e-05 5.15644531e-05 1.89550781e-05 4.48681641e-01
 4.33569336e-01 9.44238281e-06 3.41699219e-06 1.11748047e+03]: CasadiSolver.__init__() got an unexpected keyword argument 'max_num_steps'
Error for sample 2 with parameters [3.50644531e-05 6.45410156e-05 1.89550781e-05 4.48681641e-01
 4.33569336e-01 9.44238281e-06 3.41699219e-06 1.11748047e+03]: CasadiSolver.__init__() got an unexpected keyword argument 'max_num_steps'
Error for sample 3 with parameters [3.50644531e-05 5.15644531e-05 2.64355469e-05 4.48681641e-01
 4.33569336e-01 9.44238281e-06 3.41699219e-06 1.11748047e+03]: CasadiSolver.__init__() got an unexpected keyword argument 'max_num_steps'
Error for sample 4 with parameters [3.50644531e-05 5.15644531e-0

KeyboardInterrupt: 

In [31]:
# 执行灵敏度分析
Si = sobol.analyze(problem, Y)

NameError: name 'Y' is not defined